# Deploying and invoking Drift Detector Dataflow Flex template

In [2]:
%cd ../log_analyzer_flex_template

/home/jarekk/repos/drift-monitor/log_analyzer_flex_template


In [3]:
import json
from datetime import datetime
import time

## Build the Flex template docker image

In [4]:
template_name = 'log-analyzer'

project_id = !(gcloud config get-value core/project)
project_id = project_id[0]
template_image = 'gcr.io/{}/{}:latest'.format(project_id, template_name)
location = 'us-central1'

In [5]:
!gcloud builds submit --tag {template_image} .

ocal/lib/python3.5/site-packages (from apache-beam[gcp]<3,>=2.20->tensorflow_data_validation[visualization]==0.22.0) (3.0.0)
  Created wheel for absl-py: filename=absl_py-0.8.1-py3-none-any.whl size=121165 sha256=ffddc55bc7fc3ca579411d9f06dd944f8563d1e372b5c43aabe79bedb7eb7b57
  Stored in directory: /root/.cache/pip/wheels/0c/fa/af/c3217c654740d2cffc40ab4eca9da558be4bb94dd593ef0d74
  Created wheel for httplib2: filename=httplib2-0.12.0-py3-none-any.whl size=93464 sha256=af9c461b21cd52514cf0c7b2986288b0d8a2104ce70de4a6a17c31e4391202f2
  Stored in directory: /root/.cache/pip/wheels/2f/d8/97/b0a618545428f3602f565ccc1309f7cdeb8b4916ad8ec43685
  Created wheel for backcall: filename=backcall-0.1.0-py3-none-any.whl size=10413 sha256=6bac9deb2019e17b887cf85f84f7b8b34edf941d24874fe0b3cd11a481dbc951
  Stored in directory: /root/.cache/pip/wheels/cb/6f/5c/9b66a87adeb46fed2babb14407bae1271f5e41b6b34b642411
Successfully built absl-py httplib2 backcall
  Attempting uninstall: httplib2
    Found exis

## Deploy the Flex template using `gcloud`

In [6]:
flex_templates_location = 'gs://mlops-dev-workspace/flex-templates'
template_path = '{}/{}.json'.format(flex_templates_location, template_name)
metadata_file = 'metadata.json'

In [7]:
!gcloud beta dataflow flex-template build {template_path} \
  --image {template_image} \
  --sdk-language "PYTHON" \
  --metadata-file {metadata_file} \
  --image {template_image} \
  --sdk-language "PYTHON" \
  --metadata-file {metadata_file}

Successfully saved container spec in flex template file.
Template File GCS Location: gs://mlops-dev-workspace/flex-templates/log-analyzer.json
Container Spec:

{
    "image": "gcr.io/mlops-dev-env/log-analyzer:latest",
    "metadata": {
        "description": "Data drift detector Python flex template.",
        "name": "Data drift detector Python flex template",
        "parameters": [
            {
                "helpText": "A full name of the BQ request-response log table",
                "label": "Request response log table.",
                "name": "request_response_log_table",
                "regexes": [
                    "[-_.a-zA-Z0-9]+"
                ]
            },
            {
                "helpText": "A name of the AI Platform Prediction model.",
                "label": "Model name.",
                "name": "model",
                "regexes": [
                    "[-_a-zA-Z0-9]+"
                ]
            },
            {
                "helpText": "A v

## Trigger a run of the template using REST API

In [8]:
import googleapiclient.discovery

service = googleapiclient.discovery.build('dataflow', 'v1b3')

parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_classifier_logs_tf',
    'model': 'covertype_tf',
    'version': 'v3',
    'start_time': '2020-06-03T17:00:00',
    'end_time': '2020-06-03T23:00:00',
    'output_path': 'gs://mlops-dev-workspace/drift-monitor/output/covertype-tf/test',
    'schema_file': 'gs://mlops-dev-workspace/drift-monitor/schema/schema.pbtxt',
    'baseline_stats_file': 'gs://mlops-dev-workspace/drift-monitor/baseline_stats/stats.pbtxt', 
    'time_window': '60m'
}
job_name = "rr-log-analyzer-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

request = service.projects().locations().flexTemplates().launch(
    location=location,
    projectId=project_id,
    body=body)

response = request.execute()
response

{'job': {'id': '2020-06-04_14_32_24-14790456313752029810',
  'projectId': 'mlops-dev-env',
  'name': 'rr-log-analyzer-20200604-213224',
  'currentStateTime': '1970-01-01T00:00:00Z',
  'createTime': '2020-06-04T21:32:25.386283Z',
  'location': 'us-central1',
  'startTime': '2020-06-04T21:32:25.386283Z'}}

## Trigger a run of the template using REST API - Curl

In [ ]:
service_url = 'https://dataflow.googleapis.com/v1b3/projects/{}/locations/us-central1/flexTemplates:launch'.format(project_id)
headers_content = 'Content-Type: application/json'
access_token = !(gcloud auth print-access-token) 
headers_auth = 'Authorization: Bearer {}'.format(access_token[0])
parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_classifier_logs_tf',
    'model': 'covertype_tf',
    'version': 'v3',
    'start_time': '2020-05-15T00:15:00',
    'end_time': '2020-05-15T05:51:00',
    'output_path': 'gs://mlops-dev-workspace/drift_monitor/output/tf/test2',
    'schema_file': 'gs://mlops-dev-workspace/drift_monitor/schema/schema.pbtxt',
    'time_window': '60m'

}
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

json_body = json.dumps(body)

In [ ]:
!curl -X POST \
  "{service_url}" \
  -H "{headers_content}" \
  -H "{headers_auth}" \
  -d '{json_body}'

## Trigger a run of the template using `gcloud`

In [ ]:
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
g_parameters = ','.join(['{}={}'.format(key,value) for key, value in parameters.items()])

!gcloud beta dataflow flex-template run {job_name} \
--template-file-gcs-location {template_path} \
--parameters {g_parameters}